In [12]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pymorphy2
from pyaspeller import YandexSpeller
from lru import LRU
import pickle
from time import sleep
from concurrent.futures import ThreadPoolExecutor
import tarfile
import bz2
import sys
import numpy as np
with open('ill_url_ids.pickle', 'rb') as file:
    ill_url_ids = pickle.load(file)
def transform(arg, ill_url_ids):
    if arg not in ill_url_ids:
        return arg
    else:
        return ill_url_ids[arg]
class URL_info():
    def __init__(self):
        self.total_shows = 0
        self.total_clicks = 0
        self.num_first_clicks = 0
        self.num_last_clicks = 0
        self.show_poses = np.zeros((10,),dtype='int')
        self.show_poses_cl = np.zeros((10,),dtype='int')
        self.click_poses = np.zeros((10,),dtype='int')
        self.time = 0
        self.clicks_on_higher_pos = 0
        self.clicks_on_lower_pos = 0
        self.clicks_after_this = 0
        self.clicks_before_this = 0
        self.is_click_on_higher_pos = 0
        self.is_click_on_lower_pos = 0

In [2]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [3]:
all_groups

,QueryId,DocumentId
0,0,1443
1,0,5912
2,0,5963
3,0,6096
4,0,6230
...,...,...
606045,6310,497350
606046,6310,512263
606047,6310,529857
606048,6310,551291


In [4]:
q_ids = all_groups['QueryId'].unique()

In [8]:
q_docs_act = dict()

In [9]:
for qid in tqdm(q_ids):
    q_docs_act[qid] = set()
    tmp = all_groups[all_groups.QueryId==qid]
    tmp_docs = tmp['DocumentId'].unique()
    for doc in tmp_docs:
        q_docs_act[qid].add(transform(doc, ill_url_ids))

100%|██████████| 6311/6311 [00:19<00:00, 327.52it/s]


In [11]:
with open("q_docs_act.pickle", 'wb') as file:
    pickle.dump(q_docs_act, file)

In [26]:
double_q_dict = dict()

In [27]:
for key in tqdm(q_docs_act):
    double_q_dict[key] = dict()
    for elem in q_docs_act[key]:
        double_q_dict[key][elem]=URL_info()

100%|██████████| 6311/6311 [00:08<00:00, 733.38it/s]


In [28]:
double_q_dict[0]

{340485: <__main__.URL_info at 0x184e6daeaf0>,
 68106: <__main__.URL_info at 0x184e6daeb80>,
 203791: <__main__.URL_info at 0x184e6daeb50>,
 53265: <__main__.URL_info at 0x184e6daec10>,
 303636: <__main__.URL_info at 0x184e6daeb20>,
 103960: <__main__.URL_info at 0x184e4a21bb0>,
 303644: <__main__.URL_info at 0x184e4a21c10>,
 303652: <__main__.URL_info at 0x184e4a21c40>,
 107045: <__main__.URL_info at 0x184e4a21c70>,
 564772: <__main__.URL_info at 0x184e4a21b20>,
 326702: <__main__.URL_info at 0x184e4a21be0>,
 326703: <__main__.URL_info at 0x184e4a21b80>,
 194609: <__main__.URL_info at 0x184e4a21b50>,
 564788: <__main__.URL_info at 0x184e4a21af0>,
 113720: <__main__.URL_info at 0x184e4a21ac0>,
 461893: <__main__.URL_info at 0x184e4a21a90>,
 461894: <__main__.URL_info at 0x184e4a21a60>,
 461895: <__main__.URL_info at 0x184e4a21a00>,
 475209: <__main__.URL_info at 0x184e4a21a30>,
 30794: <__main__.URL_info at 0x184e4a217c0>,
 579660: <__main__.URL_info at 0x184e4a21fd0>,
 579661: <__main

In [29]:
exists = set(os.listdir('url_query'))

In [30]:
for key1 in tqdm(double_q_dict):
    if 'q_{}.pickle'.format(key1) in exists:
        with open('url_query/q_{}.pickle'.format(key1), 'rb') as file:
            obj = pickle.load(file)
            for session in obj:
                #чтсло первых кликов
                if len(session[3])>0:
                    if session[3][0] in q_docs_act[key1]:
                        double_q_dict[key1][session[3][0]].num_first_clicks+=1
                #число последних кликов  
                if len(session[3])>0:
                    if session[3][-1] in q_docs_act[key1]:
                        double_q_dict[key1][session[3][-1]].num_last_clicks+=1
                #число показов
                show_ps = dict()
                for i in range(len(session[2])):
                    show_ps[session[2][i]]=i
                    if session[2][i] in q_docs_act[key1]:
                        double_q_dict[key1][session[2][i]].total_shows+=1
                        #показы на первых 10-ти позициях
                        if i<10:
                            double_q_dict[key1][session[2][i]].show_poses[i]+=1
                #число кликов    
                for i in range(len(session[3])):
                    if session[3][i] in q_docs_act[key1]:
                        double_q_dict[key1][session[3][i]].total_clicks+=1
                        #число кликов после/до клика на этот
                        double_q_dict[key1][session[3][i]].clicks_before_this = i
                        double_q_dict[key1][session[3][i]].clicks_after_this += (len(session[3])-1-i)
                        #клики на первых 10-ти позициях
                        if show_ps[session[3][i]]<10:
                            double_q_dict[key1][session[3][i]].click_poses[show_ps[session[3][i]]]+=1
                #время
                if len(session[4])>1:
                    for i in range(len(session[4])-2):
                        if session[3][i] in q_docs_act[key1]:
                            double_q_dict[key1][session[3][i]].time += ((session[4][i+1]-session[4][i])/1000)
                pos_list = []
                for key in show_ps:
                    pos_list.append(show_ps[key])
                pos_list = np.array(pos_list)
                clicked_set = set()
                for i in range(len(session[3])):
                    if session[3][i] in q_docs_act[key1]:
                        cur_pos = show_ps[session[3][i]]
                        clicked_set.add(session[3][i])
                        a1 = np.sum(pos_list<cur_pos)
                        double_q_dict[key1][session[3][i]].clicks_on_lower_pos+=a1
                        if a1>0:
                            double_q_dict[key1][session[3][i]].is_click_on_lower_pos+=1
                        b1 = np.sum(pos_list>cur_pos)
                        double_q_dict[key1][session[3][i]].clicks_on_higher_pos+=b1
                        if b1>0:
                            double_q_dict[key1][session[3][i]].is_click_on_higher_pos+=1
                for i in range(len(session[3])):
                    if session[3][i] in clicked_set and i<10:
                        double_q_dict[key1][session[3][i]].show_poses_cl[i]+=1    

100%|██████████| 6311/6311 [12:36<00:00,  8.34it/s]  


In [31]:
with open("QDP.pickle", 'wb') as file:
    pickle.dump(double_q_dict, file)

In [32]:
2+5

7